<a href="https://colab.research.google.com/github/elgrhy/chatbot/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Twilio

In [ ]:
!pip3 install twilio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 16.8 MB/s eta 0:00:00


# Import Dependencies

In [ ]:
import datetime
import csv
from twilio.rest import Client
from datetime import datetime, timedelta

# Twilio Account Information

In [ ]:
account_sid = 'AC5d7662cb019ed4d4abed565966d521e7'
auth_token = '10be5de64f242633c480e6caf0aafaf9'
client = Client(account_sid, auth_token)

# File Name

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving appointments.csv to appointments.csv


In [ ]:
file_name = 'appointments.csv'

# List of Cities

In [ ]:
cities = ['Dubai', 'Sharjah', 'RAK']

# Process
**Get customer message and phone number**
**Check if message is about company profile change**
**Send message to customer with list of cities**
** Get customer's desired city**
**Send message to customer with available time slots**
**Get customer desired time**
**Check the appointments.csv file and give the next available date**
**Save appointment to CSV file**

**Note**
*the below number is not registered yet in the twilio*

In [ ]:
while True:
    # Get customer's message and phone number
    message = input("Enter customer's message: ")
    phone_number = input("Enter customer's phone number: ")

    # Check if message is about company profile change
    if "company profile change" in message.lower():
        # Send message to customer with list of cities
        city_list = "\n".join(cities)
        message = client.messages.create(
            body=f"Please select a city from the list below for your appointment:\n{city_list}",
            from_='whatsapp:+14155238886',
            to=f'whatsapp:{phone_number}'
        )
        print(message.sid)

        # Get customer's desired city
        city = input("Enter the city you would like to schedule the appointment: ")

        # Send message to customer with available time slots
        message = client.messages.create(
            body="Please select a time slot (10:00 AM or 12:00 PM) for your appointment.",
            from_='whatsapp:+14155238886',
            to=f'whatsapp:{phone_number}'
        )
        print(message.sid)

        # Get customer's desired time
        time = input("Enter the time you would like to schedule the appointment (HH:MM format): ")

        # Schedule appointment
        scheduled_time = datetime.strptime(time, "%H:%M")

        # check the appointments.csv file and give the next available date
        with open(file_name) as csv_file:
            reader = csv.DictReader(csv_file)
            appointments = [row for row in reader if row['city'] == city and row['time'] == scheduled_time.strftime("%H:%M")]
            appointment_date = datetime.now().date()
            while len(appointments) > 0:
                appointment_date += timedelta(days=1)
                appointments = [row for row in appointments if row['date'] != appointment_date.strftime("%Y-%m-%d")]
            print(f"Appointment scheduled for {appointment_date} at {scheduled_time} in {city}.")

            # Save appointment to CSV file
            with open(file_name, mode='a') as csv_file:
                fieldnames = ['phone_number', 'city', 'time', 'date']
                writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerow({'phone_number': phone_number, 'city': city, 'time': scheduled_time.strftime("%H:%M"), 'date': appointment_date.strftime("%Y-%m-%d")})
                break
